In [1]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F

In [2]:
class Network(nn.Module):
    
    def __init__(self):

        super().__init__()

        # sees 32, 32, 3
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(64)
        
        # sees 16, 16, 64
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(128)
        
        # sees 8, 8, 128
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.batch_norm3 = nn.BatchNorm2d(256)
        
        # sees 4, 4, 256
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.batch_norm4 = nn.BatchNorm2d(512)
        
        # will get flattened and sees 2, 2, 512
        self.fc1 = nn.Linear(512*2*2, 250)
        self.batch_norm_fc1 = nn.BatchNorm1d(250)
        
        self.fc2 = nn.Linear(250, 10)

        self.max_pool = nn.MaxPool2d(2, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        x = self.max_pool(F.relu(self.batch_norm1(self.conv1(x))))
        
        x = self.max_pool(F.relu(self.batch_norm2(self.conv2(x))))
        
        x = self.max_pool(F.relu(self.batch_norm3(self.conv3(x))))
        
        x = self.max_pool(F.relu(self.batch_norm4(self.conv4(x))))
        
        # flatten
        x = x.view(-1, 512*2*2)
        
        x = F.relu(self.batch_norm_fc1(self.fc1(x)))
        x = F.relu(self.fc2(x))
        
#         x = self.softmax(x)

        return x

In [3]:
# load CIFAR image data

In [4]:
BATCH_SIZE = 50

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# create network object
model = Network()

In [6]:
# define the optimizer and loss function

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = 'cuda:0'

In [7]:
# move to gpu
model.to(device)

Network(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=2048, out_features=250, bias=True)
  (batch_norm_fc1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=250, out_features=10, bias=True)
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dil

In [36]:
# train the model
epochs = 1
total_steps = len(trainloader)
training_losses = []
test_losses = []
train_acc = []
test_acc = []

# loop through epochs
for epoch in range(epochs):
    
    train_running_loss = 0  # track train running loss
    correct = 0
    total = 0
    
    model.train() # set to train mode
    
    # load batch images/labels
    for step, (images, labels) in enumerate(trainloader):
        
        # put data onto available device
        images = images.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
    
        outputs = model(images)  # forward pass
        _, predicted = torch.max(outputs.data, 1)  # retrieve top preds
        
        total += labels.size(0)  # add batch size
        correct += (predicted == labels).sum().item()  # calc num correct
        
        loss = criterion(outputs, labels)  # calc loss
        train_running_loss += loss.item()  # acc running loss
        
        loss.backward()   # backprop
        optimizer.step()  # forward

        if step % 100 == 0:  # print progress by iteration
        
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
            .format(epoch+1, epochs, step+1, total_steps, loss.item()))
    
    # div by num batches to get average
    epoch_train_loss = train_running_loss / len(trainloader)
                   
    print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, epochs, epoch_train_loss))
        
    # append the loss/acc after all the steps 
    training_losses.append(epoch_train_loss)
    train_acc.append(correct / total)
        
    
    # ------------------------------ #
    

    # evaluate on test data
    model.eval()
    with torch.no_grad():
        test_running_loss = 0  # track test running loss
        correct = 0
        total = 0

        for images, labels in testloader:
                   
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
                   
            loss = criterion(outputs, labels)
            test_running_loss += loss.item()
                   
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                 
    # div by num batches
    epoch_test_loss = test_running_loss / len(testloader)
                   
    print('Epoch [{}/{}], Test Loss: {:.4f}'.format(epoch+1, epochs, epoch_test_loss))
        
    # append the loss & acc after all the steps 
    test_losses.append(epoch_test_loss)
    test_acc.append(correct / total)
            
    print('Test Accuracy: {} %'.format(100 * correct / total))

Epoch [1/1], Step [1/1000], Loss: 1.3137


Traceback (most recent call last):
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  

KeyboardInterrupt: 

In [ ]:
def plot_graph(train_losses, test_losses, train_acc, test_acc):
    # plot graph
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label="Train loss")
    plt.plot(test_losses, label="Test loss")
    plt.legend(loc='best')
    plt.title("Loss vs Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss (Cross entropy)")

    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label="Train Accuracy")
    plt.plot(test_acc, label="Test Accuracy")
    plt.legend(loc='best')
    plt.title("Accuracy vs Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()
    plt.savefig('part_a_cifar_loss_acc.png')

In [ ]:
plot_graph(training_losses, test_losses, train_acc, test_acc)

In [ ]:
# # retrieve conv1 layer, convert to numpy array, put on cpu first
weights = model.conv1.weight.data.cpu()

In [ ]:
# def norm_stretch(filter):
    
#     filter = filter - torch.min(filter)
    
#     return filter / torch.max(filter)

In [ ]:
# # plot filters
# def plot_kernels(tensor, num_rows=8, num_cols=8):

#     fig = plt.figure(figsize=(num_cols,num_rows))
    
#     for i in range(tensor.shape[0]):
#         ax1 = fig.add_subplot(num_rows,num_cols,i+1)  
        
#         # need to reshape it to move channels to end
# #         reshaped = tensor[i].view(11,11,3)

#         reshaped = np.transpose(tensor.numpy(),(2,1,0))

#         norm_tensor = norm_stretch(reshaped)
        
# #         print(norm_tensor.shape)
        
# #         ax1.imshow(tensor[i])
#         ax1.imshow(norm_tensor)
#         ax1.axis('off')
#         ax1.set_xticklabels([])
#         ax1.set_yticklabels([])
    
# #     plt.subplots_adjust(wspace=0.1, hspace=0.1)
#     plt.savefig('filters_part3b.jpg')
#     plt.show()
#     plt.clf()
    

In [ ]:
# plot_kernels(weights)